<center> <h1 style="background-color:seagreen; color:white" >Load Data</h1> 

In [1]:
# Install PySpark
!pip install pyspark > /dev/null

In [2]:
# Req libraries
import os
import numpy as np
import pandas as pd

from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import *

In [3]:
# Spark Session
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('fashion-recommendations').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/20 11:55:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Load Data
transaction = spark.read.option('header','true').csv('../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')

<center> <h1 style="background-color:seagreen; color:white" >Feature Engineering</h1> 

In [5]:
# filter only 2020
sales =  transaction.withColumn('t_dat', transaction['t_dat'].cast('string'))
sales = sales.withColumn('date', from_unixtime(unix_timestamp('t_dat', 'yyyy-MM-dd')))
sales = sales.withColumn('year', year(col('date')))
sales = sales.withColumn('month', month(col('date')))

# Let's filter the data to start with
sales = sales[sales['year'] == 2020]
sales = sales[sales['month'] == 1]

transaction.unpersist()

# Prepare the dataset
sales = sales.groupby('customer_id', 'article_id').count()
sales.show(5)

23/02/20 11:56:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+--------------------+----------+-----+
|         customer_id|article_id|count|
+--------------------+----------+-----+
|0101b86c9ea4581da...|0822138002|    1|
|05b6661e851f78bc0...|0783517002|    1|
|16d13b0db3658a9fd...|0801327001|    1|
|17f52fdadc1209858...|0762846008|    1|
|1de87318baafea4da...|0788633001|    1|
+--------------------+----------+-----+
only showing top 5 rows



<center> <h1 style="background-color:seagreen; color:white" >Alternative Least Sqaure</h1>

In [6]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

<center> <h1 style="background-color:DarkKhaki; color:white" >Converting String to Index</h1>

In [7]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

indexer = [StringIndexer(inputCol=column, outputCol=column+"_index") for column in list(set(sales.columns)-set(['count'])) ]
pipeline = Pipeline(stages=indexer)
transformed = pipeline.fit(sales).transform(sales)
transformed.show(5)

23/02/20 11:59:10 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


23/02/20 11:59:32 WARN DAGScheduler: Broadcasting large task binary with size 21.3 MiB


+--------------------+----------+-----+-----------------+----------------+
|         customer_id|article_id|count|customer_id_index|article_id_index|
+--------------------+----------+-----+-----------------+----------------+
|0101b86c9ea4581da...|0822138002|    1|          14556.0|         10419.0|
|05b6661e851f78bc0...|0783517002|    1|          39354.0|          1739.0|
|16d13b0db3658a9fd...|0801327001|    1|           1107.0|          3228.0|
|17f52fdadc1209858...|0762846008|    1|          70733.0|            61.0|
|1de87318baafea4da...|0788633001|    1|           1274.0|          3222.0|
+--------------------+----------+-----+-----------------+----------------+
only showing top 5 rows



<center> <h1 style="background-color:DarkKhaki; color:white" >Creating ALS model and fitting data</h1>

In [8]:
(training,test)=transformed.randomSplit([0.8, 0.2])

als=ALS(maxIter=5,regParam=0.09,rank=25,userCol="customer_id_index",itemCol="article_id_index",ratingCol="count",coldStartStrategy="drop",nonnegative=True)
model=als.fit(training)

23/02/20 12:00:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


23/02/20 12:01:06 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


23/02/20 12:01:28 WARN DAGScheduler: Broadcasting large task binary with size 21.3 MiB


23/02/20 12:01:30 WARN DAGScheduler: Broadcasting large task binary with size 21.3 MiB


23/02/20 12:01:33 WARN DAGScheduler: Broadcasting large task binary with size 21.3 MiB


...

23/02/20 12:02:49 WARN DAGScheduler: Broadcasting large task binary with size 21.4 MiB


23/02/20 12:02:56 WARN DAGScheduler: Broadcasting large task binary with size 21.4 MiB


23/02/20 12:03:07 WARN DAGScheduler: Broadcasting large task binary with size 21.4 MiB


23/02/20 12:03:15 WARN DAGScheduler: Broadcasting large task binary with size 21.4 MiB


23/02/20 12:03:26 WARN DAGScheduler: Broadcasting large task binary with size 21.4 MiB


In [9]:
evaluator=RegressionEvaluator(metricName="rmse",labelCol="count",predictionCol="prediction")
predictions=model.transform(test)
rmse=evaluator.evaluate(predictions)
print("RMSE="+str(rmse))
predictions.show(5)

23/02/20 12:03:35 WARN DAGScheduler: Broadcasting large task binary with size 21.4 MiB
23/02/20 12:03:36 WARN DAGScheduler: Broadcasting large task binary with size 21.4 MiB


23/02/20 12:04:12 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/20 12:04:12 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


23/02/20 12:04:19 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


23/02/20 12:04:41 WARN DAGScheduler: Broadcasting large task binary with size 21.3 MiB


23/02/20 12:04:45 WARN DAGScheduler: Broadcasting large task binary with size 21.4 MiB


...

23/02/20 12:05:28 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/20 12:05:28 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


23/02/20 12:05:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


23/02/20 12:05:57 WARN DAGScheduler: Broadcasting large task binary with size 21.3 MiB


23/02/20 12:06:01 WARN DAGScheduler: Broadcasting large task binary with size 21.4 MiB


+--------------------+----------+-----+-----------------+----------------+----------+
|         customer_id|article_id|count|customer_id_index|article_id_index|prediction|
+--------------------+----------+-----+-----------------+----------------+----------+
|da624600418470043...|0708394003|    1|              1.0|          7518.0|0.85503507|
|da624600418470043...|0744291012|    2|              1.0|          1995.0| 0.9797411|
|da624600418470043...|0752814006|    1|              1.0|          1428.0| 0.9393404|
|da624600418470043...|0318066004|    1|              1.0|           597.0|0.89577645|
|da624600418470043...|0650279002|    1|              1.0|          3206.0|0.92523545|
+--------------------+----------+-----+-----------------+----------------+----------+
only showing top 5 rows



In [10]:
result = predictions.select('customer_id', 'prediction')
result.show(5)

23/02/20 12:06:03 WARN DAGScheduler: Broadcasting large task binary with size 21.4 MiB
23/02/20 12:06:04 WARN DAGScheduler: Broadcasting large task binary with size 21.4 MiB


23/02/20 12:06:40 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/20 12:06:41 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


23/02/20 12:06:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


23/02/20 12:07:10 WARN DAGScheduler: Broadcasting large task binary with size 21.3 MiB


23/02/20 12:07:13 WARN DAGScheduler: Broadcasting large task binary with size 21.4 MiB


+--------------------+----------+
|         customer_id|prediction|
+--------------------+----------+
|da624600418470043...|0.85503507|
|da624600418470043...| 0.9797411|
|da624600418470043...| 0.9393404|
|da624600418470043...|0.89577645|
|da624600418470043...|0.92523545|
+--------------------+----------+
only showing top 5 rows



In [11]:
my_pred = result.toPandas()
my_pred.to_csv('/kaggle/working/submission.csv',index=False)

23/02/20 12:07:15 WARN DAGScheduler: Broadcasting large task binary with size 21.4 MiB


23/02/20 12:07:16 WARN DAGScheduler: Broadcasting large task binary with size 21.4 MiB


23/02/20 12:07:52 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/02/20 12:07:53 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


23/02/20 12:07:59 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


23/02/20 12:08:22 WARN DAGScheduler: Broadcasting large task binary with size 21.3 MiB


23/02/20 12:08:26 WARN DAGScheduler: Broadcasting large task binary with size 21.4 MiB
